In [81]:
from selenium import webdriver
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By

# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")

# URL of the webpage containing the dropdown menu
url = "https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24"
driver.get(url)

In [143]:
years = driver.find_elements_by_class_name("DropDown_select__4pIg9")[0].text.split('\n')

table_data = []
for i in range(len(years[:2])):
    year = years[i]  
    
    season_dropdown_menu = Select(driver.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[1]/div/div/div[1]/label/div/select')) 
    season_dropdown_menu.select_by_visible_text(year)
    
    
    # Wait for the page dropdown to be present
    page_dropdown = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select'))
    )

    # Find the page dropdown element using its HTML attribute class name
    page_dropdown = Select(driver.find_elements_by_class_name("DropDown_select__4pIg9")[5])

    # Change the selected option by index, value 'All'
    page_dropdown.select_by_visible_text('All')
    
    table = driver.find_element(By.TAG_NAME, 'table')
    
    # Store the table data in a variable for looping
    tbody = table.find_element(By.TAG_NAME, 'tbody')
    rows = WebDriverWait(driver, 5).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
    )
    
    for row in rows:
        try:
            row_cells = row.find_elements(By.TAG_NAME, 'td')
            row_text = [cell.text for cell in row_cells]
            row_text.append(year)
            table_data.append(row_text) 
            print(f"Adding data for player #{row_text[0]} {row_text[1]} for the {year} season")
        except:
            # Handle StaleElementReferenceException by refinding the row
            rows = WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
            )

Adding data for player #1 Joel Embiid for the 2023-24 season
Adding data for player #2 Stephen Curry for the 2023-24 season
Adding data for player #3 Luka Doncic for the 2023-24 season
Adding data for player #4 Kevin Durant for the 2023-24 season
Adding data for player #5 Nikola Jokic for the 2023-24 season
Adding data for player #6 Giannis Antetokounmpo for the 2023-24 season
Adding data for player #7 Shai Gilgeous-Alexander for the 2023-24 season
Adding data for player #8 Donovan Mitchell for the 2023-24 season
Adding data for player #9 Jayson Tatum for the 2023-24 season
Adding data for player #10 Tyrese Maxey for the 2023-24 season
Adding data for player #11 Cam Thomas for the 2023-24 season
Adding data for player #12 Anthony Edwards for the 2023-24 season
Adding data for player #13 Desmond Bane for the 2023-24 season
Adding data for player #14 Paul George for the 2023-24 season
Adding data for player #14 Tyrese Haliburton for the 2023-24 season
Adding data for player #16 LeBron Ja

In [141]:
# Get table headers
headers = table.find_element_by_xpath('//thead').text.split(' ')
headers.append('YEAR')

In [142]:
import pandas as pd

# Getting the headers from the data array
# It is important to remove them from the data array afterwards in order to be parsed correctly by Pandas
df = pd.DataFrame(table_data, columns=headers)
df


,#,PLAYER,TEAM,GP,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,EFF,YEAR
0,2014-15,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,1,Russell Westbrook,OKC,67,34.4,28.1,9.4,22.0,42.6,1.3,...,83.5,1.9,5.4,7.3,8.6,2.1,0.2,4.4,27.7,2014-15
2,2,James Harden,HOU,81,36.8,27.4,8.0,18.1,44.0,2.6,...,86.8,0.9,4.7,5.7,7.0,1.9,0.7,4.0,27.2,2014-15
3,3,LeBron James,CLE,69,36.1,25.3,9.0,18.5,48.8,1.7,...,71.0,0.7,5.3,6.0,7.4,1.6,0.7,3.9,25.3,2014-15
4,4,Anthony Davis,NOP,68,36.1,24.4,9.4,17.6,53.5,0.0,...,80.5,2.5,7.7,10.2,2.2,1.5,2.9,1.4,30.3,2014-15
5,5,DeMarcus Cousins,SAC,59,34.1,24.1,8.4,18.1,46.7,0.0,...,78.2,3.1,9.5,12.7,3.6,1.5,1.7,4.3,27.6,2014-15
6,6,Stephen Curry,GSW,80,32.7,23.8,8.2,16.8,48.7,3.6,...,91.4,0.7,3.6,4.3,7.7,2.0,0.2,3.1,25.9,2014-15
7,7,LaMarcus Aldridge,POR,71,35.4,23.4,9.3,19.9,46.6,0.5,...,84.5,2.5,7.7,10.2,1.7,0.7,1.0,1.7,23.8,2014-15
8,8,Blake Griffin,LAC,67,35.2,21.9,8.6,17.1,50.2,0.1,...,72.8,1.9,5.7,7.6,5.3,0.9,0.5,2.3,23.7,2014-15
9,9,Kyrie Irving,CLE,75,36.4,21.7,7.7,16.5,46.8,2.1,...,86.3,0.7,2.4,3.2,5.2,1.5,0.3,2.5,19.9,2014-15
